# Dataset analysis 

In [1]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os.path
import re
import matplotlib.pyplot as plt
from typing import List
import seaborn as sns

import tikzplotlib

plt.style.use("seaborn-colorblind")
#sns.color_palette("colorblind")

In [2]:
base_path = os.path.abspath('.')
graphs_path = os.path.join(base_path, 'Analysis_graphs')
graphs_path = os.path.join(os.path.abspath('..') , 'text', 'automated_graphs')
tables_path = os.path.join(os.path.abspath('..') , 'text', 'tables')

In [3]:
def resampler(image : sitk.SimpleITK.Image, new_spacing : List[float] = None):
    """resampler to isotropic spacing

    Args:
        image (sitk.SimpleITK.Image): image to resample
        new_spacing (List[float], optional): List of three floats representing the new desired spacing. Defaults to None.

    Returns:
        sitk.SimpleITK.Image: resampled image on the new grid
    """
    
    if new_spacing is None:
        new_spacing = [1, 1, 1]

    resampler = sitk.ResampleImageFilter()
    # Nearest neighbour interpolation to avoid disturbing the labels
    # Todo: Adapt this for non-label images
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputSpacing(new_spacing)

    orig_size = np.array(image.GetSize(), dtype=np.int)
    orig_spacing = image.GetSpacing()
    new_size = np.array([x * (y / z) for x, y, z in zip(orig_size, orig_spacing, new_spacing)])
    new_size = np.ceil(new_size).astype(np.int)  # Image dimensions are in integers
    new_size = [int(s) for s in new_size]
    resampler.SetSize(new_size)

    isotropic_img = resampler.Execute(image)

    return isotropic_img

NUMPY_DIMS = {'z' : 0, 'y' : 1, 'x' : 2}
def plot_slice(itkimage : sitk.SimpleITK.Image, axis = 'z', s = 100, ax=None):
    # When converting from SimpleITK to Numpy, the order of the axis changes:
    #    SimpleITK : xyz 
    #    Numpy:      zyx
    dim = NUMPY_DIMS[axis.lower()]
    
    # resample the image on a 1 mm ✕ 1 mm ✕ 1 mm grid
    arr = sitk.GetArrayFromImage(resampler(itkimage, new_spacing = [1]*3))
    print(f'SimpleITK image size : {itkimage.GetSize()} and the numpy array : {arr.shape} *** values between {np.amin(arr)} and {np.amax(arr)} ')
    if ax is None:
        plt.imshow(arr.take(indices = s, axis = dim), cmap='gray')
        plt.grid(False)
    else:
        print(f'Add plot to axis {ax}')
        ax.imshow(arr.take(indices = s, axis = dim), cmap='gray')
        ax.grid(False)

In [4]:
df_all = pd.DataFrame(columns=['Dataset', 
                               'Patient ID',
                               'Imaging_technology', 
                               'Age', 
                               'Gender', 
                               'Anteroposterior', 
                               'Craniocaudal', 
                               'Left_right', 'Anteroposterior_delta', 'Craniocaudal_delta', 'Left_right_delta'])

## xVerSeg dataset

[link](http://lit.fe.uni-lj.si/xVertSeg/database.php)

references:

* doi:10.1109/TMI.2013.2296976
* doi:10.1109/TMI.2015.2389334




| Release date | Dataformat | number of scans |
|:-----|----------:|----------|
| OCT 5, 2015  | mhd        | 25              | 



In [5]:
data_folder = os.path.join(base_path, 'xVertSeg')
metadata_file = os.path.join(data_folder, 'metadata.csv')
metadata = pd.read_csv(metadata_file, sep=',', index_col=0)
metadata['x_dim'] = metadata.X * metadata.ΔX
metadata['y_dim'] = metadata.Y * metadata.ΔY
metadata['z_dim'] = metadata.Z * metadata.ΔZ

FileNotFoundError: [Errno 2] No such file or directory: '/home/jan/Documents/MastatThesis/Data/xVertSeg/metadata.csv'

In [ ]:
title = 'xVertSeg dataset\n'

In [ ]:
metadata.head()

In [ ]:
xVertSeg_meta = metadata[metadata.Group == 'Data1']

In [ ]:
print(xVertSeg_meta.groupby('Gender').count()['Name'])
plt.figure(figsize=(4, 8))
xVertSeg_meta.boxplot(by='Gender', column='Age', figsize=(2, 4), patch_artist=True)
plt.ylabel('Patient age [y]')
plt.ylim((0,100))
plt.xticks([1, 2], ['F (8)', 'M (7)'])
plt.title('')
#plt.suptitle(title + 'age distribution')
plt.suptitle('')

tikzplotlib.save(os.path.join(graphs_path,'xVertSeg_ageboxplot.tex'), axis_width ='5cm', axis_height ='5cm')

In [ ]:
xVertSeg_meta[['x_dim', 'y_dim', 'z_dim']].plot(kind = 'box', patch_artist=True)
plt.ylabel('dimension [mm]')
plt.xticks([1, 2, 3], ['left-right (x)', 'anteroposterior (y)', 'craniocaudal (z)'])
#plt.title('Distribution of the xVertSeg volume dimensions')
tikzplotlib.save(os.path.join(graphs_path,'xVertSeg_dimensionboxplot.tex'), axis_width ='9cm', axis_height ='5cm')

xVertSeg_meta[['x_dim', 'y_dim', 'z_dim']].mean()

Todo: Investigate very short sample further!

In [ ]:
image_name = os.path.join(data_folder, 'Data1', 'images', 'image002.mhd')
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 110, ax=ax1)
ax1.set_xlabel('Left-right axis (x) [mm]')
ax1.set_ylabel('Anteroposterior axis (y) [mm]')

plot_slice(itkimage, axis = 'x', s = 200, ax=ax2)
ax2.set_xlabel('Anteroposterior axis (y)[mm]')
ax2.set_ylabel('Craniocaudal axis (z) [mm]')

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

plt.savefig(os.path.join(graphs_path,'xVertSeg_image002.png'))
tikzplotlib.save(os.path.join(graphs_path,'xVertSeg_image002.tex'), axis_width ='9cm', axis_height ='5cm', textsize = 9)

In [ ]:
plt.hist(sitk.GetArrayFromImage(itkimage).flatten(), bins = 15) 
tikzplotlib.save(os.path.join(graphs_path,'xVertSeg_image002_intensityhistogram.tex'))

Conclusion: For the xVerSeg dataset:

 * x = Left-right axis  
 * y = Anteroposterior axis  
 * z = Craniocaudam axis

In [ ]:
scores_file = os.path.join(data_folder, 'Data1', 'scores', 'scores.csv')
scores = pd.read_csv(scores_file, header=1)
cases = {0:'normal',1:'wedge', 2:'biconcave', 3:'crush'}
grad = {0:'normal', 1:'mild', 2:'moderate', 3:'severe'}
scores = scores.replace({f'L{i} case' : cases for i in [1,2,3,4,5]} )
scores = scores.replace({f'L{i} grade' : grad for i in [1,2,3,4,5]})
scores.set_index('Spine image').transpose().to_latex(os.path.join(tables_path,'xVertSeg_pathologies.tex'), index=True)
scores.head()

In [ ]:
counts = pd.DataFrame()
for i in [1,2,3,4,5]:
    counts[f'L{i}'] = scores[f'L{i} case'].value_counts().astype(int)

In [ ]:
counts = counts.fillna(0).astype(int)
counts['Total'] = counts.sum(axis=1)
counts.to_latex(os.path.join(tables_path,'xVertSeg_pathologies_summary.tex'), index=True)
counts

In [ ]:
metadata.columns

In [ ]:
metadata['Dataset'] = 'xVertSeg'
metadata['Imaging_technology'] = 'CT'
metadata['Patient ID'] = ['xVertSeg_{}'.format(i) for i in metadata.index]


df_all = df_all.append(metadata[['x_dim', 
                                 'y_dim', 
                                 'z_dim', 
                                 'Dataset', 
                                 'Age', 
                                 'Gender',
                                'Imaging_technology',
                                'Patient ID', 'ΔX', 'ΔY', 'ΔZ']].rename(columns = {'x_dim' : 'Left_right',
                                                   'y_dim' : 'Anteroposterior',
                                                   'z_dim' : 'Craniocaudal', 'ΔY':'Anteroposterior_delta', 'ΔZ': 'Craniocaudal_delta', 'ΔX' : 'Left_right_delta'}))
df_all

## USiegen

doi.org/10.1111/cgf.12343]

In [ ]:
data_folder = os.path.join(base_path, 'USiegen')
title = 'USiegen dataset\n'

In [ ]:
metadata_file = os.path.join(data_folder, 'metadata.csv')
USiegen_meta = pd.read_csv(metadata_file, sep=',').rename(columns={"Sex": "Gender"})
USiegen_meta['Dataset'] = USiegen_meta['Dataset'].str.encode('utf-8').astype('string')
USiegen_meta['Dataset'] = USiegen_meta.Dataset.apply(lambda x : x.split("\'")[1])
USiegen_meta['Gender'] = USiegen_meta.Gender.astype('category')

In [ ]:
USiegen_meta['Patient ID'] = USiegen_meta['Patient ID'].apply(lambda x : f'USiegen_{int(x)}')
USiegen_meta

In [ ]:
image_name = os.path.join(data_folder, 'SpineDatasets', 'AKa3.dcm')
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 40, ax=ax1)
ax1.set_xlabel('Anteroposterior\naxis (x) [mm]')
ax1.set_ylabel('Craniocaudal\naxis (y) [mm]')
ax1.set_xticks([])
ax1.set_yticks([])

plot_slice(itkimage, axis = 'y', s = 40, ax=ax2)
ax2.set_xlabel('Anteroposterior axis (x) [mm]')
ax2.set_ylabel('Left-right axis\n(z) [mm]')
ax2.set_xticks([])
ax2.set_yticks([])

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

plt.savefig(os.path.join(graphs_path,'USiegen_Aka3.png'))
tikzplotlib.save(os.path.join(graphs_path,'USiegen_Aka3.tex'), axis_width ='9cm', axis_height ='5cm', textsize = 9)

In [ ]:
image_name = os.path.join(data_folder, 'SpineSegmented', 'AKa3', 'L1.mha')
print(image_name)
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 40, ax=ax1)
ax1.set_xlabel('Anteroposterior axis (x) [mm]')
ax1.set_ylabel('Craniocaudal axis (y) [mm]')

plot_slice(itkimage, axis = 'y', s = 40, ax=ax2)
ax2.set_xlabel('Anteroposterior axis (x) [mm]')
ax2.set_ylabel('Left-right axis (z) [mm]')

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

print(f'unique values : {np.unique(sitk.GetArrayFromImage(itkimage))}')

In [ ]:
plt.hist(sitk.GetArrayFromImage(itkimage).flatten(), bins = 15) 
plt.title("Intensity histogram AKa3") 
tikzplotlib.save(os.path.join(graphs_path,'USiegen_AKa3_intensityhistogram.tex'))

Conclusion: For the USiegen dataset:

 * x = Anteroposterior axis  
 * y = Craniocaudal axis
 * z = Left-right axis

In [ ]:
spines = os.path.join(data_folder, 'SpineDatasets')

ls_name = []
ls_x = []
ls_y = []
ls_z = []

ls_Δx = []
ls_Δy = []
ls_Δz = []

for file in os.listdir(spines):
    itkimage = sitk.ReadImage(os.path.join(spines, file))
    # print(f'file : {file} - size : {itkimage.GetSize()} - spacing : {itkimage.GetSpacing()}')
    ls_name.append(file.split('.')[0])
    x, y, z = itkimage.GetSize()
    Δx, Δy, Δz = itkimage.GetSpacing()
    ls_x.append(x)
    ls_y.append(y)
    ls_z.append(z)
    ls_Δx.append(Δx)
    ls_Δy.append(Δy)
    ls_Δz.append(Δz)
    
USiegen_sizes = pd.DataFrame(data={
    'Dataset' : ls_name,
    'X':ls_x,
    'Y':ls_y,
    'Z':ls_z,
    'ΔX':ls_Δx,
    'ΔY':ls_Δy,
    'ΔZ':ls_Δz
})

USiegen_sizes['Dataset'] = USiegen_sizes['Dataset'].str.encode('utf-8').astype('string')

USiegen_sizes['Dataset'] = USiegen_sizes.Dataset.apply(lambda x : x.split("\'")[1])

In [ ]:
USiegen_meta = USiegen_sizes.set_index('Dataset').join(USiegen_meta.set_index('Dataset'))

USiegen_meta['x_dim'] = USiegen_meta.X * USiegen_meta.ΔX
USiegen_meta['y_dim'] = USiegen_meta.Y * USiegen_meta.ΔY
USiegen_meta['z_dim'] = USiegen_meta.Z * USiegen_meta.ΔZ

USiegen_meta

In [ ]:
USiegen_meta.describe()

In [ ]:
print(USiegen_meta.groupby('Gender').count()['SB'])

USiegen_meta.boxplot(by='Gender', column='Age', patch_artist=True)
plt.ylabel('Patient age [y]')
plt.ylim((0,100))
plt.xticks([1, 2], ['F(11)', 'M(6)'])
plt.title('')
plt.grid(linestyle = ':', alpha = .5)
plt.suptitle('')

tikzplotlib.save(os.path.join(graphs_path,'USiegen_ageboxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 9)

In [ ]:
plt.figure()

plt.subplot(2,1,1)

USiegen_meta[['x_dim', 'y_dim', 'z_dim']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('dimension [mm]')
plt.xticks([1, 2, 3], ['', '', ''])
plt.title('')

plt.subplot(2,1,2)

USiegen_meta[['ΔX', 'ΔY', 'ΔZ']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('spacing [mm]')
plt.xticks([1, 2, 3], ['anteroposterior (x)', 'craniocaudal (y)', 'left-right (z)'])
plt.title('')
plt.tight_layout()
tikzplotlib.save(os.path.join(graphs_path,'USiegen_dimensionboxplot.tex'), axis_width ='9cm', axis_height ='7cm', textsize = 9)

Conclusion: For the USiegen dataset:

 * x = Anteroposterior axis  
 * y = Craniocaudal axis
 * z = Left-right axis

In [ ]:
metadata = USiegen_meta
metadata['Dataset'] = 'USiegen'
metadata['Imaging_technology'] = 'CT'

df_all = df_all.append(metadata[['x_dim', 
                                 'y_dim', 
                                 'z_dim', 
                                 'Dataset', 
                                 'Age', 
                                 'Gender',
                                'Imaging_technology', 'Patient ID', 'ΔX', 'ΔY', 'ΔZ']].rename(columns = {'z_dim' : 'Left_right',
                                                   'x_dim' : 'Anteroposterior',
                                                   'y_dim' : 'Craniocaudal', 'ΔX':'Anteroposterior_delta', 'ΔY': 'Craniocaudal_delta', 'ΔZ' : 'Left_right_delta'}))

## MyoSegment_TUM
OSF Sara Schlaeger

    DOI: 10.1186/s12891-019-2528-x

In [ ]:
data_folder = os.path.join(base_path, 'OSF_Sarah_Schlaeger')
title = 'OSF dataset\n'

In [ ]:
OSF_meta = pd.read_excel(os.path.join(data_folder, 'Table1.xlsx'))
OSF_meta = OSF_meta.rename(columns = {'sex' : 'Gender', 'age [y]' : 'Age', 'height[m]':'Height'
                                      , 'BMI [kg/m^2]' : 'BMI', 'weight [kg]' : 'Weight', 'ID' : 'Dataset'})

sex_encode = {'female' : 'F', 'male' : 'M'}
OSF_meta = OSF_meta = OSF_meta.replace({'Gender' : sex_encode} )
OSF_meta['Gender'] = OSF_meta['Gender'].astype('category')
OSF_meta['Patient ID'] = [f'OSF_{i}' for i in OSF_meta.index]

In [ ]:
OSF_meta.head()

In [ ]:
print(OSF_meta.groupby('Gender').count()['Dataset'])
OSF_meta.boxplot(by='Gender', column='Age', patch_artist=True)
plt.ylabel('Patient age [y]')
plt.ylim((0,100))
plt.xticks([1, 2], ['F(39)', 'M(15)'])
plt.title('')

plt.suptitle('')

tikzplotlib.save(os.path.join(graphs_path,'OSF_ageboxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 9)

In [ ]:
OSF_meta.boxplot(by='Gender', column='BMI', patch_artist=True)
plt.ylabel('Patient BMI')
plt.xticks([1, 2], ['F(39)', 'M(15)'])
plt.title('')
plt.grid(False)
plt.suptitle('')

tikzplotlib.save(os.path.join(graphs_path,'OSF_BMIboxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 9)

In [ ]:
OSF_meta.boxplot(by='Gender', column='Height', patch_artist=True)
plt.ylabel('Patient Height [m]')
plt.xticks([1, 2], ['F(39)', 'M(15)'])
plt.title('')

plt.suptitle('')

tikzplotlib.save(os.path.join(graphs_path,'OSF_Heightboxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 9)

In [ ]:
image_name = os.path.join(data_folder, '02', '02', 'vertebrae', 'T2star.dcm')
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 40, ax=ax1)
ax1.set_xlabel('Anteroposterior\naxis (x) [mm]')
ax1.set_ylabel('Craniocaudal\naxis (y) [mm]')

plot_slice(itkimage, axis = 'y', s = 40, ax=ax2)
ax2.set_xlabel('Anteroposterior\naxis (x) [mm]')
ax2.set_ylabel('Left-right\naxis (z) [mm]')

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

plt.savefig(os.path.join(graphs_path,'OSF_02.png'))
tikzplotlib.save(os.path.join(graphs_path,'OSF_02.tex'), axis_width ='9cm', axis_height ='5cm', textsize = 9)

In [ ]:
plt.hist(sitk.GetArrayFromImage(itkimage).flatten(), bins = 15) 
plt.title("Intensity histogram image 02") 
tikzplotlib.save(os.path.join(graphs_path,'OSF_02_intensityhistogram.tex'))

In [ ]:
ls_name = []
ls_x = []
ls_y = []
ls_z = []

ls_Δx = []
ls_Δy = []
ls_Δz = []

for i in range(1,54):
    file_name = os.path.join(data_folder, f'{i:02d}', f'{i:02d}', 'vertebrae', 'T2star.dcm')
    itkimage = sitk.ReadImage(file_name)
    ls_name.append(i)
    x, y, z = itkimage.GetSize()
    Δx, Δy, Δz = itkimage.GetSpacing()
    ls_x.append(x)
    ls_y.append(y)
    ls_z.append(z)
    ls_Δx.append(Δx)
    ls_Δy.append(Δy)
    ls_Δz.append(Δz)
    
OSF_sizes = pd.DataFrame(data={
    'Dataset' : ls_name,
    'X':ls_x,
    'Y':ls_y,
    'Z':ls_z,
    'ΔX':ls_Δx,
    'ΔY':ls_Δy,
    'ΔZ':ls_Δz
})


In [ ]:
OSF_meta = OSF_sizes.set_index('Dataset').join(OSF_meta.set_index('Dataset'))

OSF_meta['x_dim'] = OSF_meta.X * OSF_meta.ΔX
OSF_meta['y_dim'] = OSF_meta.Y * OSF_meta.ΔY
OSF_meta['z_dim'] = OSF_meta.Z * OSF_meta.ΔZ

OSF_meta.head()

In [ ]:
OSF_meta.describe()

In [ ]:
plt.figure()

plt.subplot(2,1,1)

OSF_meta[['x_dim', 'y_dim', 'z_dim']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('dimension [mm]')
plt.xticks([1, 2, 3], ['', '', ''])
plt.title('')

plt.subplot(2,1,2)

OSF_meta[['ΔX', 'ΔY', 'ΔZ']].plot(kind = 'box', ax=plt.gca())
plt.ylabel('spacing [mm]')
plt.xticks([1, 2, 3], ['anteroposterior (x)', 'craniocaudal (y)', 'left-right (z)'])
plt.title('')
plt.tight_layout()
tikzplotlib.save(os.path.join(graphs_path,'OSF_dimensionboxplot.tex'), axis_width ='9cm', axis_height ='7cm', textsize = 9)

Conclusion: For the USiegen dataset:

 * x = Anteroposterior axis  
 * y = Craniocaudal axis
 * z = Left-right axis

In [ ]:
metadata = OSF_meta
metadata['Dataset'] = 'MyoSegment_TUM'
metadata['Imaging_technology'] = 'MRI'

df_all = df_all.append(metadata[['x_dim', 
                                 'y_dim', 
                                 'z_dim', 
                                 'Dataset', 
                                 'Age', 
                                 'Gender',
                                'Imaging_technology', 
                                 'Patient ID', 'ΔX', 'ΔY', 'ΔZ']].rename(columns = {'z_dim' : 'Left_right',
                                                   'x_dim' : 'Anteroposterior',
                                                   'y_dim' : 'Craniocaudal', 'ΔX':'Anteroposterior_delta', 'ΔY': 'Craniocaudal_delta', 'ΔZ' : 'Left_right_delta'}))

## Zenodo

https://zenodo.org/record/22304#.YE-XE_4o82B

In [ ]:
data_folder = os.path.join(base_path, 'Zenodo')

In [ ]:
ls_name = []
ls_x = []
ls_y = []
ls_z = []

ls_Δx = []
ls_Δy = []
ls_Δz = []

for i in range(1, 23):
    file_name = os.path.join(data_folder, f'Img_{i:02d}.nii')
    itkimage = sitk.ReadImage(file_name)
    ls_name.append(i)
    x, y, z = itkimage.GetSize()
    Δx, Δy, Δz = itkimage.GetSpacing()
    ls_x.append(x)
    ls_y.append(y)
    ls_z.append(z)
    ls_Δx.append(Δx)
    ls_Δy.append(Δy)
    ls_Δz.append(Δz)

Zenodo_meta = pd.DataFrame(data={
    'Dataset' : ls_name,
    'X':ls_x,
    'Y':ls_y,
    'Z':ls_z,
    'ΔX':ls_Δx,
    'ΔY':ls_Δy,
    'ΔZ':ls_Δz
})

Zenodo_meta['x_dim'] = Zenodo_meta.X * Zenodo_meta.ΔX
Zenodo_meta['y_dim'] = Zenodo_meta.Y * Zenodo_meta.ΔY
Zenodo_meta['z_dim'] = Zenodo_meta.Z * Zenodo_meta.ΔZ

Zenodo_meta['Patient ID'] = [f'Zenodo_{i}' for i in Zenodo_meta.index]

In [ ]:
image_name = os.path.join(data_folder, f'Img_02.nii')
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 190, ax=ax1)
ax1.set_xlabel('left-right axis (x) [mm]')
ax1.set_ylabel('Anteroposterior axis (y) [mm]')

plot_slice(itkimage, axis = 'x', s = 40, ax=ax2)
ax2.set_xlabel('Anteroposterior axis (x) [mm]')
ax2.set_ylabel('Craniocaudal axis (z) [mm]')

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

plt.savefig(os.path.join(graphs_path,'PLoS_img02.png'))
tikzplotlib.save(os.path.join(graphs_path,'PLoS_img02.tex'), axis_width ='9cm', axis_height ='5cm', textsize = 9)

In [ ]:
image_name = os.path.join(data_folder, f'Img_02_Labels.nii')
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 190, ax=ax1)
ax1.set_xlabel('left-right axis (x) [mm]')
ax1.set_ylabel('Anteroposterior axis (y) [mm]')

plot_slice(itkimage, axis = 'x', s = 40, ax=ax2)
ax2.set_xlabel('Anteroposterior axis (x) [mm]')
ax2.set_ylabel('Craniocaudal axis (z) [mm]')

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

In [ ]:
plt.hist(sitk.GetArrayFromImage(itkimage).flatten(), bins = 15) 
plt.title("Intensity histogram image 4564688") 
tikzplotlib.save(os.path.join(graphs_path,'PLoS_img02_intensityhistogram.tex'))

In [ ]:
plt.figure()

plt.subplot(2,1,1)

Zenodo_meta[['x_dim', 'y_dim', 'z_dim']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('dimension [mm]')
plt.xticks([1, 2, 3], ['', '', ''])
plt.title('')

plt.subplot(2,1,2)

Zenodo_meta[['ΔX', 'ΔY', 'ΔZ']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('spacing [mm]')
plt.xticks([1, 2, 3], ['left-right (x)', 'anteroposterior (y)', 'craniocaudal (z)'])
plt.title('')
plt.tight_layout()
tikzplotlib.save(os.path.join(graphs_path,'PLoS_dimensionboxplot.tex'), axis_width ='5cm', axis_height ='15cm', textsize = 9)

In [ ]:
Zenodo_meta.describe()

In [ ]:
metadata = Zenodo_meta
metadata['Dataset'] = 'PLoS'
metadata['Imaging_technology'] = 'MRI'
metadata['Age'] = 'Unknown'
metadata['Gender'] = 'Unknown'

print(f'Dimension before appending PLoS : {df_all.shape[0]}')

df_all = df_all.append(metadata[['x_dim', 
                                 'y_dim', 
                                 'z_dim', 
                                 'Dataset', 
                                 'Age', 
                                 'Gender',
                                'Imaging_technology', 'Patient ID', 'ΔX', 'ΔY', 'ΔZ']].rename(columns = {'x_dim' : 'Left_right',
                                                   'y_dim' : 'Anteroposterior',
                                                   'z_dim' : 'Craniocaudal', 'ΔY':'Anteroposterior_delta', 'ΔZ': 'Craniocaudal_delta', 'ΔX' : 'Left_right_delta'}))
print(f'Dimension after appending PLoS : {df_all.shape[0]}')

## University of Washington

source : 

In [ ]:
data_folder = os.path.join(base_path, 'UWSpineCT-selected')

metadata_file = os.path.join(data_folder, 'UWSpineCT-meta-data.xlsx')

In [ ]:
UW_meta = pd.read_excel(metadata_file, sheet_name='allSpine-out-DH-orig', header=[0,1])
UW_meta = UW_meta.drop(UW_meta.columns[2], axis=1)

UW_meta

In [ ]:
UW_meta_patients = UW_meta.iloc[:,[0,1,6,7,8]]
UW_meta_patients = UW_meta_patients.dropna(axis=0)
UW_meta_patients.columns = UW_meta_patients.columns.droplevel(0)
#UW_meta_patients['Patient ID'] = UW_meta_patients['Patient ID'].apply(lambda x : f'UW_{int(x)}')
UW_meta_patients.head()

In [ ]:
print(UW_meta_patients.groupby('Gender').count()['Patient ID'])
UW_meta_patients.boxplot(by='Gender', column='Age', patch_artist=True)
plt.ylabel('Patient age (y)')
plt.ylim((0,100))
plt.xticks([1, 2], ['F(54)', 'M(71)'])
plt.title('')

plt.suptitle('')

tikzplotlib.save(os.path.join(graphs_path,'UW_ageboxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 9)

In [ ]:
UW_meta_scans = UW_meta.iloc[:,[0, 1, 2, 3, 4]]
UW_meta_scans.columns = UW_meta_scans.columns.droplevel(0)
UW_meta_scans = UW_meta_scans.rename(columns={'Scan ID' : 'Dataset'})
#UW_meta_scans['Patient ID'] = UW_meta_scans['Patient ID'].apply(lambda x : f'UW_{int(x)}')
UW_meta_scans.shape

In [ ]:
print(
    'University of Washington dataset contains {} scans of {} different patients'.format(UW_meta_scans.shape[0], UW_meta_scans['Patient ID'].nunique())
)

In [ ]:
file_paths = list()
for f in os.walk(data_folder):
    folder = f[0]
    files = f[2]
    for file in files:
        if file.endswith('.nii.gz') and 'spine-test-data' not in folder:
            file_paths.append(
                os.path.join(folder, file)
            )

ls_name = []
ls_x = []
ls_y = []
ls_z = []

ls_Δx = []
ls_Δy = []
ls_Δz = []

get_ID = re.compile(r'(\d+).nii.gz$')

for file_name in file_paths:
    itkimage = sitk.ReadImage(file_name)
    # print(int(get_ID.findall(file_name)[0]))
    ls_name.append(int(get_ID.findall(file_name)[0]))
    x, y, z = itkimage.GetSize()
    Δx, Δy, Δz = itkimage.GetSpacing()
    ls_x.append(x)
    ls_y.append(y)
    ls_z.append(z)
    ls_Δx.append(Δx)
    ls_Δy.append(Δy)
    ls_Δz.append(Δz)

UW_meta2 = pd.DataFrame(data={
    'Dataset' : ls_name,
    'X':ls_x,
    'Y':ls_y,
    'Z':ls_z,
    'ΔX':ls_Δx,
    'ΔY':ls_Δy,
    'ΔZ':ls_Δz
})

UW_meta2['x_dim'] = UW_meta2.X * UW_meta2.ΔX
UW_meta2['y_dim'] = UW_meta2.Y * UW_meta2.ΔY
UW_meta2['z_dim'] = UW_meta2.Z * UW_meta2.ΔZ

UW_meta_scans = UW_meta_scans.set_index('Dataset').join(UW_meta2.set_index('Dataset'))

In [ ]:

image_name = file_paths[3]
itkimage = sitk.ReadImage(image_name)

fig, (ax1, ax2) = plt.subplots(1,2)

plot_slice(itkimage, axis = 'z', s = 40, ax=ax1)
ax1.set_xlabel('left-right axis (x) [mm]')
ax1.set_ylabel('Anteroposterior axis (y) [mm]')

plot_slice(itkimage, axis = 'y', s = 70, ax=ax2)
ax2.set_xlabel('Anteroposterior axis (y) [mm]')
ax2.set_ylabel('Craniocaudal axis (z) [mm]')

ax1.set_anchor('S')
ax2.set_anchor('S')

plt.tight_layout()

plt.savefig(os.path.join(graphs_path,'UW_4564688.png'))
tikzplotlib.save(os.path.join(graphs_path,'UW_4564688.tex'), axis_width ='9cm', axis_height ='5cm', textsize = 9)

In [ ]:
plt.hist(sitk.GetArrayFromImage(itkimage).flatten(), bins = 15) 
plt.title("Intensity histogram image 4564688") 
tikzplotlib.save(os.path.join(graphs_path,'UW_4564688_intensityhistogram.tex'))

Conclusion: For the xVerSeg dataset:

 * x = Left-right axis  
 * y = Anteroposterior axis  
 * z = Craniocaudal axis

In [ ]:
UW_meta_scans

In [ ]:
plt.figure()

plt.subplot(2,1,1)

UW_meta_scans[['x_dim', 'y_dim', 'z_dim']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('dimension [mm]')
plt.xticks([1, 2, 3], ['', '', ''])
plt.title('')

plt.subplot(2,1,2)

UW_meta_scans[['ΔX', 'ΔY', 'ΔZ']].plot(kind = 'box', ax=plt.gca(), patch_artist=True)
plt.ylabel('spacing [mm]')
plt.xticks([1, 2, 3], ['left-right (x)', 'anteroposterior (y)', 'craniocaudal (z)'])
plt.title('')
plt.tight_layout()
tikzplotlib.save(os.path.join(graphs_path,'UW_dimensionboxplot.tex'), axis_width ='9cm', axis_height ='7cm', textsize = 9)

In [ ]:
UW_meta_scans['Patient ID'] = UW_meta_scans['Patient ID'].astype('int16')
UW_meta_patients['Patient ID'] = UW_meta_patients['Patient ID'].astype('int16')

print(f'scans length : {UW_meta_scans.shape[0]} and patients length : {UW_meta_patients.shape[0]}')

metadata = UW_meta_patients[['Gender', 
                          'Age', 
                          'Patient ID']].set_index('Patient ID').join(UW_meta_scans[['Patient ID',
                                                            'x_dim', 'y_dim', 'z_dim', 'ΔX', 'ΔY', 'ΔZ']].set_index('Patient ID'), 
                                              how='left',  lsuffix='patients', rsuffix='scans').reset_index(drop=False)

print(metadata.columns)

metadata['Dataset'] = 'UWashington'
metadata['Imaging_technology'] = 'CT'
metadata['Patient ID'] = metadata['Patient ID'].apply(lambda x : f'UW_{int(x)}')

print(f'before adding the UW set : {df_all.shape[0]}')

df_all = df_all.append(metadata[['x_dim', 
                                 'y_dim', 
                                 'z_dim', 
                                 'Dataset', 
                                 'Age', 
                                 'Gender',
                                'Imaging_technology',
                                'Patient ID', 'ΔX', 'ΔY', 'ΔZ']].rename(columns = {'x_dim' : 'Left_right',
                                                   'y_dim' : 'Anteroposterior',
                                                   'z_dim' : 'Craniocaudal', 'ΔY':'Anteroposterior_delta', 'ΔZ': 'Craniocaudal_delta', 'ΔX' : 'Left_right_delta'}))

print(f'after adding the UW set : {df_all.shape[0]}')

In [ ]:
df_all.Dataset.unique

## Overview of all datapoints

In [ ]:
df_all['Age'] = df_all['Age'].replace('Unknown', np.nan).astype('int16', errors = 'ignore')
df_all_countGender = df_all.drop_duplicates(
    subset=['Patient ID']).groupby('Gender').agg({'Dataset': 'count'}).rename(columns = {'Dataset' : 'amount'})
df_all_avgAgeGender = df_all.drop_duplicates(
    subset=['Patient ID']).dropna(how='any').groupby('Gender').agg({'Age': 'mean', 'Dataset': 'count'}).rename(columns = {'Age' : 'Average age', 'Dataset' : 'amount'})
df_all_avgAgeDataset = df_all.drop_duplicates(
    subset=['Patient ID']).groupby(['Dataset', 'Gender']).agg({'Age': 'mean', 'Gender': 'count'}).rename(columns = {'Gender' : 'amount'})

datasets = [df_all, df_all_countGender,df_all_avgAgeGender,df_all_avgAgeDataset, df_all.drop_duplicates(subset=['Patient ID'])]
names = ['all', 'countGender', 'avgAgeGender', 'avgAgeDataset', 'all_patients']

with pd.ExcelWriter(os.path.join(tables_path,'overview.xlsx')) as writer:
    for df, name in zip(datasets, names):
        df.to_excel(writer, sheet_name=name)
        df.to_latex(os.path.join(tables_path,f'df_all_{name}.tex'))

print(df_all.groupby('Imaging_technology').count()['Dataset'])

In [ ]:
df_all_countGender

In [ ]:
df_all.groupby('Dataset').count()

In [ ]:
df_all[df_all.Dataset == 'PLoS']

In [ ]:
#df_all = df_all.replace('Unknown',np.nan)
# df_all = df_all.dropna(axis=0, how='any')
sns.set(font_scale=1.1)

sns.set_style('white')

df_all['Age'] = df_all['Age'].astype('int16', errors='ignore')

df_patients = df_all.drop_duplicates(subset=['Patient ID'])
g= sns.catplot(x='Gender', col = 'Dataset', y='Age', data=df_patients, palette = ['grey', 'lightgrey'], kind='box')
#g.set_ylabel("patient age [y]")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.savefig(os.path.join(graphs_path,'AllDatset_ageboxplot.png'))
tikzplotlib.save(os.path.join(graphs_path,'AllDataset_ageboxplot.tex'), axis_width ='9cm', axis_height ='5cm', textsize = 9)

In [ ]:
g= sns.catplot(x='dimension', 
               row = 'Dataset', 
               y='length [mm]', 
               data=pd.melt(df_all,
                            id_vars = ['Dataset'], var_name='dimension',
                            value_vars=['Anteroposterior', 'Craniocaudal', 'Left_right'], value_name='length [mm]'),
              palette = 'colorblind', kind='box')

plt.tight_layout()
tikzplotlib.save(os.path.join(graphs_path,'AllDataset_DimensionsBoxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 9)
plt.savefig(os.path.join(graphs_path,'AllDataset_DimensionsBoxplot.png'))

In [ ]:
g= sns.catplot(x='dimension', 
               row = 'Dataset', 
               y='spacing [mm]', 
               data=pd.melt(df_all,
                            id_vars = ['Dataset'], var_name='dimension',
                            value_vars=['Anteroposterior_delta', 'Craniocaudal_delta', 'Left_right_delta'], value_name='spacing [mm]').replace({'dimension':{'Anteroposterior_delta' : 'Anteroposterior', 'Craniocaudal_delta':'Craniocaudal' , 'Left_right_delta' : 'Left-right'}}),
              palette = 'colorblind', kind='box')
plt.tight_layout()
tikzplotlib.save(os.path.join(graphs_path,'AllDataset_SpacingBoxplot.tex'), axis_width ='5cm', axis_height ='5cm', textsize = 15)
plt.savefig(os.path.join(graphs_path,'AllDataset_SpacingBoxplot.png'))

In [ ]:
for group in df_all.groupby('Dataset'):
    print(group)